# MRI Image Generation using Least Squares GAN (LSGAN)

This notebook trains an LSGAN model on middle sagittal slices extracted from 3D MRI volumes to generate new, synthetic MRI images.

## Overview
1. Install required packages
2. Load and preprocess 3D MRI data
3. Extract middle slices from 3D volumes
4. Train LSGAN model
5. Generate new MRI images

## 1. Install Required Packages

In [ ]:
# Install necessary packages
!pip install nibabel
!pip install matplotlib
!pip install scikit-image

## 2. Import Libraries and Set Parameters

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

import nibabel as nib
import matplotlib.pyplot as plt
import skimage.transform  # Properly import skimage.transform
from glob import glob

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check if CUDA is available
cuda = True if torch.cuda.is_available() else False
device = torch.device("cuda" if cuda else "cpu")
print(f"Using {device} device")

# Create directories for saving results
os.makedirs("mri_images", exist_ok=True)
os.makedirs("models", exist_ok=True)

# Parameters
batch_size = 64
lr = 0.0002
b1 = 0.5
b2 = 0.999
latent_dim = 128
n_epochs = 500
img_size = 128  # Resize MRI slices to this size
channels = 1  # Grayscale images
sample_interval = 200

Using cuda device


## 3. Data Loading and Preprocessing

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
from glob import glob

class MRIDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.image_paths = sorted(glob(os.path.join(data_dir, "*.png")))
        self.transform = transform
        
        print(f"Found {len(self.image_paths)} PNG slices")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Convert to grayscale

        if self.transform:
            image = self.transform(image)

        return image
        
# Define transforms: convert to tensor & normalize to [-1, 1]
data_transform = transforms.Compose([
    # transforms.Resize((128, 128)),  # Resize if needed
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Load dataset
dataset = MRIDataset(data_dir="PATH/TO/DATA/FOLDER", transform=data_transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Display a sample
import matplotlib.pyplot as plt

if len(dataset) > 0:
    sample_slice = dataset[0].squeeze().numpy()
    plt.figure(figsize=(6, 6))
    plt.imshow(sample_slice, cmap='gray')
    plt.title('Sample Coronal Slice (Y-axis)')
    plt.axis('off')
    plt.show()
else:
    print("No data found!")


## 4. Define Generator and Discriminator Models

In [5]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = img_size // 4
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = img_size // 2 ** 4
        self.adv_layer = nn.Linear(128 * ds_size ** 2, 1)

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)
        return validity

## 5. Initialize Models and Optimizers

In [7]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

# Move models to GPU if available
if cuda:
    generator.cuda()
    discriminator.cuda()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Define loss function (least squares loss)
adversarial_loss = torch.nn.MSELoss()

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

# For easier GPU handling
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

def save_model_checkpoint(model, filename):
    """Save model checkpoint"""
    torch.save(model.state_dict(), filename)
    print(f"Model saved to {filename}")

## 6. Training Loop

In [ ]:
# Function to visualize generated images during training
def visualize_generator_output(generator, fixed_noise, epoch, batches_done):
    generator.eval()
    with torch.no_grad():
        gen_imgs = generator(fixed_noise)
        # Rescale from [-1, 1] to [0, 1]
        gen_imgs = 0.5 * gen_imgs + 0.5
        
        # Save grid of generated images
        save_image(gen_imgs.data[:25], f"mri_images/epoch_{epoch}_batch_{batches_done}.png", nrow=5, normalize=False)
        
        # Display images
        if batches_done % (sample_interval * 5) == 0:
            plt.figure(figsize=(10, 10))
            plt.imshow(make_grid(gen_imgs[:25], nrow=5).cpu().numpy().transpose(1, 2, 0), cmap='gray')
            plt.axis('off')
            plt.title(f"Generated MRI Images (Epoch {epoch}, Batch {batches_done})")
            plt.show()
    generator.train()

# Create fixed noise vector for visualization
fixed_noise = Variable(Tensor(np.random.normal(0, 1, (25, latent_dim))))

# Training loop
for epoch in range(n_epochs):
    for i, real_imgs in enumerate(dataloader):
        # Configure input
        real_imgs = Variable(real_imgs.type(Tensor))
        batch_size = real_imgs.shape[0]
        
        # Adversarial ground truths
        valid = Variable(Tensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (batch_size, latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Calculate generator loss (LSGAN)
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = 0.5 * (real_loss + fake_loss)

        d_loss.backward()
        optimizer_D.step()

        # Print progress
        batches_done = epoch * len(dataloader) + i
        if i % 10 == 0:
            print(
                f"[Epoch {epoch}/{n_epochs}] [Batch {i}/{len(dataloader)}] "
                f"[D loss: {d_loss.item():.6f}] [G loss: {g_loss.item():.6f}]"
            )

        # Visualize generator output
        if batches_done % sample_interval == 0:
            visualize_generator_output(generator, fixed_noise, epoch, batches_done)
    
    # Save model checkpoints at the end of each epoch
    if epoch % 10 == 0:
        save_model_checkpoint(generator, f"models/generator_epoch_{epoch}.pt")
        save_model_checkpoint(discriminator, f"models/discriminator_epoch_{epoch}.pt")

# Save final models
save_model_checkpoint(generator, "models/generator_final.pt")
save_model_checkpoint(discriminator, "models/discriminator_final.pt")

## 7. Generate New MRI Images (Inference)

In [ ]:
def generate_new_mri_images(generator, num_images=10, latent_dim=128):
    """Generate new MRI images using the trained generator"""
    # Set model to evaluation mode
    generator.eval()
    
    # Create random noise vectors
    noise = Variable(Tensor(np.random.normal(0, 1, (num_images, latent_dim))))
    
    # Generate images
    with torch.no_grad():
        generated_images = generator(noise)
        # Rescale from [-1, 1] to [0, 1]
        generated_images = 0.5 * generated_images + 0.5
    
    # Plot generated images
    plt.figure(figsize=(15, 5))
    for i in range(num_images):
        plt.subplot(2, 5, i+1)
        plt.imshow(generated_images[i].cpu().squeeze().numpy(), cmap='gray')
        plt.axis('off')
        plt.title(f"Generated MRI #{i+1}")
    
    plt.tight_layout()
    plt.show()
    
    # Save the generated images
    save_image(generated_images, "mri_images/final_generated_samples.png", nrow=5, normalize=False)
    
    return generated_images

# Generate new MRI images
generated_samples = generate_new_mri_images(generator, num_images=10)
print("MRI Generation Complete!")

## 8. Inference with Saved Model (For Future Use)

In [ ]:
def load_generator_and_infer(model_path, num_images=5):
    """Load a saved generator model and use it to generate new MRI images"""
    # Initialize a new generator
    loaded_generator = Generator()
    
    # Load the saved parameters
    loaded_generator.load_state_dict(torch.load(model_path))
    
    # Move to appropriate device
    if cuda:
        loaded_generator.cuda()
    
    # Generate images
    generated_samples = generate_new_mri_images(loaded_generator, num_images, latent_dim)
    
    return generated_samples

# Example usage (commented out - use this after training is complete)
generated_samples = load_generator_and_infer("models/generator_final.pt", num_images=10)